In [ ]:
import cv2
from keras.models import Sequential,load_model,Model
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Subtract,add,maximum,minimum
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras import backend as K
from keras.models import model_from_json
import pandas as pd
import numpy as np
import os
import random
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage
import time
from skimage.color import rgb2gray
%matplotlib inline
import tensorflow as tf

In [ ]:
def crop_image(img_path):
    # extract pre-trained face detector
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

    # load color (BGR) image
    img = cv2.imread(img_path)
    # convert BGR image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # find faces in image
    faces = face_cascade.detectMultiScale(gray)
    cropped = img
    for (x,y,w,h) in faces:
        cropped = img[y:y+h,x:x+w]
        cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
    return cropped

In [ ]:
data_dir = './lfw-deepfunneled'

In [ ]:
# HYPERPARAMETERS 
BATCHES_IN_A_EPOCH = 20
EPOCHS = 100
BATCH_SIZE = 32
IMAGE_SIZE = 150
LR = 0.001   # LEARNING RATE

In [ ]:
from keras import backend as K

input_shape = (IMAGE_SIZE,IMAGE_SIZE,1)
left_input = Input(input_shape)
right_input = Input(input_shape)

model = Sequential()
model.add(Conv2D(32,(2,2),padding = 'same',input_shape=(IMAGE_SIZE,IMAGE_SIZE,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))
model.add(Conv2D(64,(2,2),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))
model.add(Conv2D(128,(2,2),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))
model.add(Conv2D(256,(2,2),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(3,3))
# model.add(Conv2D(512,(2,2),padding = 'same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(3,3))
model.add(Flatten())
model.add(Dense(512,activation='sigmoid'))

encoded_l = model(left_input)
encoded_r = model(right_input)

maxi = maximum([encoded_l,encoded_r])
mini = minimum([encoded_l,encoded_r])

subtracted = Subtract()([maxi,mini])

fc1 = Dense(1024,activation='relu')(subtracted)
out = Dense(1,activation='sigmoid')(fc1)

NetWork = Model(input=[left_input,right_input],outputs = out)
NetWork.summary()

In [ ]:
NetWork.compile(optimizer=Adam(lr=LR),loss='binary_crossentropy')

In [ ]:
multiple= []   # LIST STORING THE NAMES OF THE PERSON CONTAINING MULTIPLE IMAGES
count = 0   # STORES THE COUNT OF TOTAL IMAGES FOR THE PERSON HAVING MULTIPLE IMAGES
classes = 0
for name in os.listdir(data_dir):
    classes+=1
    images= []
    images = os.listdir(data_dir+'/'+name)
    if(len(images)>1):
        count+=len(images)
        multiple.append(name)

print(f"Total no of persons: {classes}")
print("Total no of people having multiple images: {} ".format(len(multiple)))
print(f"Total count of images of such person is: {count}")

In [ ]:
def get_matching_pairs(batch_size):
#     print("GETTING MATCHING PAIRS")
    l_image=[]
    r_image=[]
    for i in range(0,batch_size):
        num = random.randint(0,len(multiple)-1)
        name = multiple[num]
        total = []
        total = os.listdir('./lfw-deepfunneled/'+name)
        img1 = random.randint(0,len(total)-1)
        img2 = random.randint(0,len(total)-1)
        if(img1==img2):
            img2 = (img1+1)%(len(total))
        
        img1_path = './lfw-deepfunneled/'+name+'/'+total[img1]
        img2_path = './lfw-deepfunneled/'+name+'/'+total[img2]
        l_img = crop_image(img1_path)
        r_img = crop_image(img2_path)
        l_img = (rgb2gray(l_img))
        r_img = (rgb2gray(r_img))
        l_img = skimage.transform.resize(l_img, (IMAGE_SIZE,IMAGE_SIZE,1))
        r_img = skimage.transform.resize(r_img, (IMAGE_SIZE,IMAGE_SIZE,1))
        l_img = np.asarray(l_img)
        r_img = np.asarray(r_img)
        
        l_image.append(l_img)
        r_image.append(r_img)


    l_image = np.asarray(l_image)
    r_image = np.asarray(r_image)
    return l_image,r_image

In [ ]:
def get_non_matching_pairs(batch_size):
#     print("GETTING NON MATCHING PAIRS...")
    l_image=[]
    r_image=[]
    for i in range(0,batch_size):
        num1 = random.randint(0,len(multiple)-1)
        num2 = random.randint(0,len(multiple)-1)
        if(num1==num2):
            num2 = (num1+1)%(len(multiple))
        name1 = multiple[num1]
        name2 = multiple[num2]
        total1 = []
        total2 = []
        total1 = os.listdir('./lfw-deepfunneled/'+name1)
        total2 = os.listdir('./lfw-deepfunneled/'+name2)
        img1 = random.randint(0,len(total1)-1)
        img2 = random.randint(0,len(total2)-1)
        img1_path = './lfw-deepfunneled/'+name1+'/'+total1[img1]
        img2_path = './lfw-deepfunneled/'+name2+'/'+total2[img2]
        l_img = crop_image(img1_path)
        r_img = crop_image(img2_path)
        l_img = (rgb2gray(l_img))
        r_img = (rgb2gray(r_img))

        l_img = skimage.transform.resize(l_img, (IMAGE_SIZE,IMAGE_SIZE,1))
        r_img = skimage.transform.resize(r_img, (IMAGE_SIZE,IMAGE_SIZE,1))
        l_img = np.asarray(l_img)
        r_img = np.asarray(r_img)

        l_image.append(l_img)
        r_image.append(r_img)


    l_image = np.asarray(l_image)
    r_image = np.asarray(r_image)
    return l_image,r_image

In [ ]:
import time
losses = []
def train(epochs=20):
    print("Training Begins...")
    for epoch in range(1,epochs+1):
        loss = 0
        start_time = time.time()
        for j in range(0,BATCHES_IN_A_EPOCH//4):
            l,r = get_non_matching_pairs(BATCH_SIZE)      # non-matching pairs
            target = np.ones(BATCH_SIZE)                  # 1s
            non_matching_loss1 = NetWork.train_on_batch([l,r],target)
            
            l,r = get_matching_pairs(BATCH_SIZE)          # matching pairs
            target = np.zeros(BATCH_SIZE)                 # 0s
            matching_loss1 = NetWork.train_on_batch([l,r],target)
            
            l,r = get_non_matching_pairs(BATCH_SIZE)      # non-matching pairs
            target = np.ones(BATCH_SIZE)                  # 1s
            non_matching_loss2 = NetWork.train_on_batch([l,r],target)
            
            l,r = get_matching_pairs(BATCH_SIZE)          # matching pairs
            target = np.zeros(BATCH_SIZE)                 # 0s
            matching_loss2 = NetWork.train_on_batch([l,r],target)
            
            loss = matching_loss1+matching_loss2+non_matching_loss1+non_matching_loss2

        losses.append(loss)
        print(f"Epoch {epoch}/{epochs}:\nDuration:{time.time()-start_time}\nTraining_loss = {np.mean(loss):.6f}")

In [ ]:
train(5)

In [ ]:
THRESHOLD=0.5

In [ ]:
def test_non_matching_pairs():
    l_image=[]
    r_image=[]
    done=[]
    img1=[]
    img2=[]
    for name1 in os.listdir('./TestFaces'):
        done.append(name1)
        img1 = os.listdir('./TestFaces/'+name1)
        for name2 in os.listdir('./TestFaces'):
            if name2 in done:
                continue
            img2 = os.listdir('./TestFaces/'+name2)
            for i in range(0,3):
                img1_path = './TestFaces/'+name1+'/'+img1[i]
                img2_path = './TestFaces/'+name2+'/'+img2[i]
                l_img = crop_image(img1_path)
                r_img = crop_image(img2_path)
                l_img = (rgb2gray(l_img))
                r_img = (rgb2gray(r_img))
                
                plt.subplot(1,3,1)
                plt.imshow(l_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
                plt.subplot(1,3,3)
                plt.imshow(r_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
                plt.show()

                l_img = skimage.transform.resize(l_img, (IMAGE_SIZE,IMAGE_SIZE,1))
                r_img = skimage.transform.resize(r_img, (IMAGE_SIZE,IMAGE_SIZE,1))
                l_img = np.asarray(l_img)
                r_img = np.asarray(r_img)
                l_image.append(l_img)
                r_image.append(r_img)


    l_image = np.asarray(l_image)
    r_image = np.asarray(r_image)
    return l_image,r_image

In [ ]:
l,r = test_non_matching_pairs()
print(l.shape)
print(r.shape)

In [ ]:
test_result = NetWork.predict([l,r])
acc=0
for num in test_result:
    if(num>=THRESHOLD):
        acc+=1
print(f'ACCURACY: {(acc/l.shape[0])*100:.3f}%')
print("\n\n")
print(test_result)

In [ ]:
def test_matching_pairs():
    l_image=[]
    r_image=[]
    for names in os.listdir('./TestFaces'):
        print("NAME: "+names)
        for i in range(0,3):
            img1_path = './TestFaces/'+names+'/'+str((i%3)+1)+'.jpg'
            img2_path = './TestFaces/'+names+'/'+str(((i+1)%3)+1)+'.jpg'
            l_img = crop_image(img1_path)
            r_img = crop_image(img2_path)
            l_img = (rgb2gray(l_img))
            r_img = (rgb2gray(r_img))
    
            print("Images: "+str((i%3)+1)+"\t\t\t\t"+str(((i+1)%3)+1))
            plt.subplot(1,3,1)
            plt.imshow(l_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
            plt.subplot(1,3,3)
            plt.imshow(r_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
            plt.show()
            
            l_img = skimage.transform.resize(l_img, (IMAGE_SIZE,IMAGE_SIZE,1))
            r_img = skimage.transform.resize(r_img, (IMAGE_SIZE,IMAGE_SIZE,1))
            l_img = np.asarray(l_img)
            r_img = np.asarray(r_img)
            l_image.append(l_img)
            r_image.append(r_img)
    l_image = np.asarray(l_image)
    r_image = np.asarray(r_image)
    return l_image,r_image

In [ ]:
l,r = test_matching_pairs()
print(l.shape)
print(r.shape)

In [ ]:
test_result = NetWork.predict([l,r])
acc=0
for num in test_result:
    if(num<THRESHOLD):
        acc+=1
print(f'ACCURACY: {(acc/l.shape[0])*100:.3f}%')
print("\n\n")
print(test_result)

In [ ]:
os.chdir('./try4(BEST)')
from keras.models import model_from_json
model_json = NetWork.to_json()
with open("model4d.json","w") as json_file:
    json_file.write(model_json)
NetWork.save_weights("model4d.h5")
print("Model Saved")

In [ ]:
json_file = open('try4(BEST)/model4d.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("try4(BEST)/model4d.h5")
print("Model Loaded")

In [ ]:
os.chdir('../')

In [ ]:
loaded_model.summary()

In [ ]:
ll=[]
l_img = crop_image('./FACES/img.jpeg')
l_img = (rgb2gray(l_img))
plt.imshow(l_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
l_img = skimage.transform.resize(l_img, (IMAGE_SIZE,IMAGE_SIZE,1))
l_img = np.asarray(l_img)
ll.append(l_img)
ll = np.asarray(ll)
ll.shape

In [ ]:
rr=[]
r_img = crop_image('./FACES/PIC2.jpg')
r_img = (rgb2gray(r_img))
plt.imshow(r_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
r_img = skimage.transform.resize(r_img, (IMAGE_SIZE,IMAGE_SIZE,1))
r_img = np.asarray(r_img)
rr.append(r_img)
rr = np.asarray(rr)
rr.shape

In [ ]:
loaded_model.predict([ll,rr])             # close to zero means same person